In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Digital Twin Demo: Confocal Microscopy\n",
    "\n",
    "[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_GITHUB_USERNAME/confocal_microscopy-copilot/blob/main/notebooks/digital_twin_demo.ipynb)\n",
    "\n",
    "This notebook demonstrates the `DigitalTwin` in `copilot/digital_twin.py`:\n",
    "\n",
    "- Load the twin and configs from this repo\n",
    "- Generate synthetic confocal stacks (colloidal gel vs nuclei presets)\n",
    "- Run the parameter recommender on a noisy stack\n",
    "- Show how to plug in a real dataset instead of synthetic data\n",
    "\n",
    "You can run this in VS Code (Python + Jupyter) or directly in Google Colab using the badge above.  \n",
    "Make sure the notebook lives in `notebooks/digital_twin_demo.ipynb` so the Colab link stays valid.[web:66][web:67][web:68]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "#@title Setup: clone repo (for Colab) or adjust path\n",
    "import os, sys, math\n",
    "\n",
    "IN_COLAB = 'google.colab' in sys.modules\n",
    "\n",
    "if IN_COLAB:\n",
    "    # Clone your GitHub repo when running in Colab\n",
    "    if not os.path.exists('confocal_microscopy-copilot'):\n",
    "        !git clone https://github.com/Abhishek-Gupta-GitHub/confocal_microscopy-copilot.git\n",
    "    os.chdir('confocal_microscopy-copilot')\n",
    "\n",
    "# Add repo root to Python path (VS Code + Colab)\n",
    "repo_root = os.path.abspath(os.path.join(os.getcwd()))\n",
    "if repo_root not in sys.path:\n",
    "    sys.path.insert(0, repo_root)\n",
    "\n",
    "print(\"Using repo root:\", repo_root)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Import the digital twin and utilities\n",
    "\n",
    "This cell imports the `DigitalTwin` class and sets up a few helper functions for plotting.  \n",
    "If you change the structure of `copilot/digital_twin.py`, only this section should need small edits."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "from copilot.digital_twin import DigitalTwin\n",
    "\n",
    "plt.rcParams['figure.figsize'] = (5, 4)\n",
    "plt.rcParams['image.cmap'] = 'magma'\n",
    "\n",
    "def show_frame(stack, frame_idx=0, z_idx=None, title=\"\"):\n",
    "    \"\"\"Simple helper to visualize a single frame or z-slice.\"\"\"\n",
    "    stack = np.asarray(stack)\n",
    "    if stack.ndim != 4:\n",
    "        raise ValueError(\"Expected stack with shape (T, Z, Y, X)\")\n",
    "    t = min(frame_idx, stack.shape[0] - 1)\n",
    "    if z_idx is None:\n",
    "        z = stack.shape[1] // 2\n",
    "    else:\n",
    "        z = min(z_idx, stack.shape[1] - 1)\n",
    "    img = stack[t, z]\n",
    "    plt.imshow(img)\n",
    "    plt.colorbar(label=\"Intensity\")\n",
    "    plt.title(title or f\"frame={t}, z={z}\")\n",
    "    plt.axis(\"off\")\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Define microscope + sample presets\n",
    "\n",
    "Here we define two presets:\n",
    "\n",
    "- **Colloidal gel**: many small particles, subdiffusive viscoelastic motion, near-wall hindrance.\n",
    "- **Cell nuclei**: fewer, larger blobs (approximated by higher brightness and fewer particles), more viscous motion.\n",
    "\n",
    "These are simple dictionaries that you can later turn into YAML/JSON configs."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Shared base microscope config\n",
    "base_microscope = {\n",
    "    \"na\": 1.3,\n",
    "    \"wavelength_nm\": 550.0,\n",
    "    \"immersion_ri\": 1.33,\n",
    "    \"sample_ri\": 1.33,\n",
    "    \"pixel_size_xy_um\": 0.1,\n",
    "    \"z_step_um\": 0.2,\n",
    "    \"img_shape_xyz\": (32, 96, 96),  # (Z, Y, X)\n",
    "    \"psf_mode\": \"gaussian_confocal\",\n",
    "    # optional acquisition-ish parameters\n",
    "    \"exposure_ms\": 5.0,\n",
    "    \"pinhole_factor\": 1.0,\n",
    "    # defaults for bleaching config when not overridden\n",
    "    \"z_att_um\": 50.0,\n",
    "    \"bleach_tau_s\": 80.0,\n",
    "    \"noise_std\": 5.0,\n",
    "}\n",
    "\n",
    "colloidal_sample = {\n",
    "    \"n_particles\": 400,\n",
    "    \"box_size_um\": (30.0, 30.0, 30.0),\n",
    "    \"motion_model\": \"viscoelastic_near_wall\",\n",
    "    \"D\": 0.1,\n",
    "    \"D0\": 0.15,\n",
    "    \"alpha\": 0.6,   # subdiffusive\n",
    "    \"z_wall_um\": 0.0,\n",
    "    \"z_cut_um\": 4.0,\n",
    "    \"brightness\": 1.0,\n",
    "}\n",
    "\n",
    "nuclei_sample = {\n",
    "    \"n_particles\": 80,\n",
    "    \"box_size_um\": (40.0, 40.0, 40.0),\n",
    "    \"motion_model\": \"viscous\",\n",
    "    \"D\": 0.05,\n",
    "    \"D0\": 0.05,\n",
    "    \"alpha\": 1.0,\n",
    "    \"z_wall_um\": 0.0,\n",
    "    \"z_cut_um\": 6.0,\n",
    "    \"brightness\": 3.0,\n",
    "}\n",
    "\n",
    "twin = DigitalTwin()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Generate a synthetic colloidal gel stack\n",
    "\n",
    "This section simulates a 3D+time stack for the colloidal gel preset and visualizes one frame.  \n",
    "We also show a simple intensity profile vs depth to see the attenuation + PSF blur qualitatively."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "n_frames = 20\n",
    "dt = 0.1  # seconds\n",
    "\n",
    "stack_colloid, pos_colloid = twin.simulate(\n",
    "    microscope_config=base_microscope,\n",
    "    sample_config=colloidal_sample,\n",
    "    n_frames=n_frames,\n",
    "    dt=dt,\n",
    ")\n",
    "\n",
    "print(\"Colloidal stack shape:\", stack_colloid.shape)\n",
    "show_frame(stack_colloid, frame_idx=0, z_idx=None, title=\"Colloidal gel: frame 0, mid z\")\n",
    "\n",
    "# Depth profile at t=0: mean intensity vs z\n",
    "depth_profile = stack_colloid[0].mean(axis=(1, 2))\n",
    "plt.plot(depth_profile)\n",
    "plt.xlabel(\"z index\")\n",
    "plt.ylabel(\"Mean intensity\")\n",
    "plt.title(\"Depth profile (colloidal gel, t=0)\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Generate a synthetic nuclei stack\n",
    "\n",
    "Here we switch only the sample config to mimic cell nuclei.  \n",
    "The same `DigitalTwin.simulate` call is used, showing how presets can share the same forward model."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "stack_nuclei, pos_nuclei = twin.simulate(\n",
    "    microscope_config=base_microscope,\n",
    "    sample_config=nuclei_sample,\n",
    "    n_frames=n_frames,\n",
    "    dt=dt,\n",
    ")\n",
    "\n",
    "print(\"Nuclei stack shape:\", stack_nuclei.shape)\n",
    "show_frame(stack_nuclei, frame_idx=0, z_idx=None, title=\"Cell nuclei: frame 0, mid z\")\n",
    "\n",
    "# Depth profile at t=0 for comparison\n",
    "depth_profile_n = stack_nuclei[0].mean(axis=(1, 2))\n",
    "plt.plot(depth_profile_n)\n",
    "plt.xlabel(\"z index\")\n",
    "plt.ylabel(\"Mean intensity\")\n",
    "plt.title(\"Depth profile (nuclei, t=0)\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Parameter recommender on a noisy stack\n",
    "\n",
    "Now we pretend that the colloidal stack is a noisy real dataset.  \n",
    "We call `recommend_parameters` to search over a small grid of exposure times and pinhole factors and return a human-readable recommendation.\n",
    "\n",
    "In your UI, you would run this on the **real** stack and then let the LLM explain the recommendation in context of the physics JSON summary."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "suggestion = twin.recommend_parameters(\n",
    "    real_stack=stack_colloid,\n",
    "    base_microscope_config=base_microscope,\n",
    "    sample_config=colloidal_sample,\n",
    "    n_frames_test=6,\n",
    "    dt=dt,\n",
    "    exposure_candidates_ms=[5.0, 10.0, 20.0],\n",
    "    pinhole_factors=[1.0, 0.7],\n",
    ")\n",
    "\n",
    "print(\"Recommendation:\\n\", suggestion[\"recommendation\"])\n",
    "print(\"\\nBase metrics:\")\n",
    "for k, v in suggestion[\"base_metrics\"].items():\n",
    "    print(f\"  {k}: {v:.3f}\")\n",
    "\n",
    "print(\"\\nBest candidate:\")\n",
    "for k, v in suggestion[\"best_candidate\"].items():\n",
    "    if isinstance(v, dict):\n",
    "        print(f\"  {k}:\")\n",
    "        for kk, vv in v.items():\n",
    "            print(f\"    {kk}: {vv:.3f}\")\n",
    "    else:\n",
    "        print(f\"  {k}: {v}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Using a real confocal dataset instead of synthetic\n",
    "\n",
    "This section sketches how to replace the synthetic stack by a real dataset loaded from your `data/` folder (e.g. OME-TIFF).  \n",
    "You can integrate this with your existing IO utilities or bioformats readers.\n",
    "\n",
    "For the hackathon, it is enough to:\n",
    "\n",
    "- Load a small 3D+time stack into a NumPy array `real_stack` with shape `(T, Z, Y, X)`.\n",
    "- Provide a matching `microscope_config` dictionary (pixel size, z-step, etc.).\n",
    "- Call `recommend_parameters` exactly as above.\n",
    "\n",
    "Below is placeholder code you can adapt to your actual data loader."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from pathlib import Path\n",
    "\n",
    "def load_example_real_stack(path: str) -> np.ndarray:\n",
    "    \"\"\"Placeholder: replace with a real TIFF/OME loader.\n",
    "\n",
    "    For now, this just returns a subset of the synthetic colloidal stack\n",
    "    as a stand-in for a real dataset.\n",
    "    \"\"\"\n",
    "    # TODO: integrate with your real IO (e.g. tifffile or aicspylibczi)\n",
    "    _ = path  # unused placeholder\n",
    "    return stack_colloid.copy()\n",
    "\n",
    "# Example usage\n",
    "data_path = \"data/example_real_stack.tif\"  # adjust to your dataset\n",
    "real_stack = load_example_real_stack(data_path)\n",
    "print(\"Real stack (placeholder) shape:\", real_stack.shape)\n",
    "\n",
    "# Reuse base_microscope as an approximate config; tweak fields as needed.\n",
    "real_suggestion = twin.recommend_parameters(\n",
    "    real_stack=real_stack,\n",
    "    base_microscope_config=base_microscope,\n",
    "    sample_config=colloidal_sample,  # or a better-matched sample config\n",
    "    n_frames_test=6,\n",
    "    dt=dt,\n",
    ")\n",
    "\n",
    "print(\"\\nReal-data suggestion (placeholder IO):\")\n",
    "print(real_suggestion[\"recommendation\"])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Next steps and integration notes\n",
    "\n",
    "- Move the microscope and sample presets into YAML/JSON config files and load them here to keep the notebook clean.\n",
    "- Feed the `recommend_parameters` output into your LLM explainer to turn the recommendation into a user-facing narrative.\n",
    "- Connect this notebook to your GUI (Gradio) by reusing the same `DigitalTwin.simulate` and `recommend_parameters` calls under the hood.\n",
    "\n",
    "In VS Code, you can run this notebook with the Jupyter extension.  \n",
    "In Colab, the badge at the top will open it directly from GitHub.[web:64][web:66][web:68]"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
